# 🎨 ArtExplorer: Part 1 - The Data & AI Foundation

This notebook serves as the complete data engineering and AI model creation pipeline for the ArtExplorer project, a submission for the BigQuery AI Hackathon.

### **1. Purpose of This Notebook**

The goal of this notebook is to perform all the necessary backend setup within Google Cloud Platform. It is designed to be run **once** to prepare the data and models that the final application will use. The key processes automated here are:

1.  **Data Ingestion & Enrichment**: Downloading the raw MET Museum dataset, filtering it, and enriching it with detailed metadata via the MET Collection API.
2.  **Cloud Storage Upload**: Uploading the cleaned and enriched data to a Google Cloud Storage bucket.
3.  **BigQuery Ingestion**: Loading the data from GCS into a structured BigQuery table.
4.  **AI Model Creation**: Creating the BQML remote models for text embedding (`text-embedding-004`) and generative storytelling (`gemini-2.0-flash-001`).
5.  **Feature Engineering**: Generating the final text embeddings for each artwork using `ML.GENERATE_EMBEDDING` and storing them in a new, vector-search-ready table.

    **A Note on Data Enrichment Results**
    
    You will notice that while the initial filtering step identifies over 2,000 potential artworks, the final enriched dataset contains a smaller number (e.g., 81 artworks in a test run with LIMIT_ROWS=100).This is an expected and intentional outcome of our robust data enrichment process. The enrich_artwork_data function calls the live MET Collection API for each artwork. Some of these API calls may fail due to various real-world factors, such as missing data for a specific Object ID on the API server or transient network issues. Our pipeline is designed to be resilient: instead of failing the entire process, it gracefully skips any artwork for which enrichment fails, ensuring that only complete, high-quality data is included in the final dataset. This demonstrates a practical approach to handling real-world, imperfect data sources.

### **2. End-to-End Technical Architecture**

This notebook builds the foundational components (the data pipeline) of our technical architecture. The full architecture, including the interactive application, is as follows:
``` 
[MET Museum API] -> [Python Data Enrichment (Kaggle Notebook)] -> [Google Cloud Storage]
                                                                          |
                                                                          v
                                                             [BigQuery AI Engine]
                                                              /                  \
[ML.GENERATE_EMBEDDING] -> [Vector Table] -> [VECTOR_SEARCH] <---> [Interactive App (UI)] <---> [ML.GENATE_TEXT (Gemini)]
``` 

Upon successful execution of this notebook, all necessary GCP assets will be ready to serve the front-end application.

### **3. Next Steps**

Once this pipeline has been successfully run, you can proceed to the main application notebook, `ArtExplorer_AI_Docent_(Application).ipynb`, to experience the interactive AI Docent.

### **CELL 1: Environment Setup**

This cell prepares the Kaggle environment by installing and configuring the necessary Python libraries. It ensures all Google Cloud clients are up-to-date and resolves a known version conflict with the `protobuf` library.

In [1]:
# CELL 1: SETUP - Install and Configure Libraries
# ===================================================================
print("--- [1/6] Setting up the environment: Installing and configuring libraries... ---")

# Upgrade core Google Cloud libraries...
!pip install --upgrade -q google-cloud-bigquery google-cloud-storage google-cloud-bigquery-storage google-cloud-aiplatform 

# Downgrade the protobuf library...
!pip install -q protobuf==3.20.3

print("✅ Library setup complete.")
print("ℹ️ A kernel restart may be recommended for the library changes to take full effect.")

--- [1/6] Setting up the environment: Installing and configuring libraries... ---
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.3/259.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.6/293.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.8/160.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.0/322.0 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 72.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-automl 1.0.1 requires google-api-core[grpc]<2.0.0dev,>=1.14.0, but you have google-api-core 2.25.1 which is incompatible.
google-cloud-translate 3.12.1 requires protobuf!=3.20

### **CELL 2: Authentication & Global Configuration**

This cell is the foundational setup block for the entire notebook. It performs three critical tasks:

1.  **Imports Libraries**: All necessary Python libraries for the project are imported here.
2.  **GCP Authentication**: It authenticates with GCP using a secure Service Account key stored in Kaggle Secrets.
3.  **Global Configuration**: It defines all essential global variables (Project ID, Bucket Name, etc.) in one central location.

---
> 🔐 **Important Prerequisite: Connecting Kaggle Secrets**
>
> To run this notebook, you must first have a Kaggle Secret named `GCP_CREDENTIALS` containing your GCP Service Account JSON key.
>
> **On the first run**, Kaggle will automatically detect that this notebook needs access to the secret and will display a pop-up window asking you to **"Attach Secret"**.
>
> **Please click the "Attach" button in the pop-up to proceed.** You may need to run this cell a second time after attaching the secret for the authentication to complete successfully.

In [2]:
# ===================================================================
# CELL 2: AUTHENTICATION & GLOBAL CONFIGURATION
# ===================================================================
print("--- [2/6] Importing libraries, authenticating, and setting configuration... ---")

# --- 1. Import All Project Libraries ---
import os
import json
import pandas as pd
import requests
from google.cloud import bigquery, storage
from google.oauth2 import service_account
from kaggle_secrets import UserSecretsClient
from tqdm.auto import tqdm
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# --- 2. GCP Service Account Authentication ---
# Disable Kaggle's default GCP integration to prevent any conflicts with our
# explicit Service Account authentication method.
os.environ['KAGGLE_DISABLE_GCP_INTEGRATION'] = 'true'

# Load the service account key from Kaggle Secrets.
user_secrets = UserSecretsClient()
gcp_key_string = user_secrets.get_secret("GCP_CREDENTIALS")
gcp_key_json = json.loads(gcp_key_string)

# Define the necessary permission scopes to grant full access to GCP services.
scopes = ['https://www.googleapis.com/auth/cloud-platform']

# Create the final credentials object using the key and defined scopes.
credentials = service_account.Credentials.from_service_account_info(
    gcp_key_json,
    scopes=scopes
)
print("✅ Service Account authentication successful.")

# --- 3. Global Project Configuration ---
# All key parameters for the project are defined here.
# ❗ Ensure these values match your GCP environment setup.
PROJECT_ID = "semantic-art-explorer"
YOUR_BUCKET_NAME = "semantic-art-explorer-20250829"
DATASET_ID = "art_dataset"  # Using a consistent, clean name.
REGION = "us-central1"

# --- 4. Data Processing Limit (for Development & Testing) ---
# Set to an integer (e.g., 100) for quick tests, or None to process the entire dataset.
LIMIT_ROWS = None

print(f"✅ Configuration loaded for Project ID: {PROJECT_ID}")

# --- 5. Initialize GCP Clients ---
# Instantiate clients for BigQuery and Cloud Storage, passing the explicit
# credentials object to ensure stable and correct authentication.
bq_client = bigquery.Client(project=PROJECT_ID, credentials=credentials)
storage_client = storage.Client(project=PROJECT_ID, credentials=credentials)
print("✅ BigQuery and GCS clients initialized successfully.")

--- [2/6] Importing libraries, authenticating, and setting configuration... ---
✅ Service Account authentication successful.
✅ Configuration loaded for Project ID: semantic-art-explorer
✅ BigQuery and GCS clients initialized successfully.


### **CELL 3: Data Pipeline - Fetch, Enrich, and Upload to GCS**

This cell constitutes the primary data engineering pipeline for the project. It automates the process of acquiring, cleaning, and preparing the artwork data for use in our BigQuery AI models. The pipeline consists of four main stages:

1.  **Download**: Fetches the initial `MetObjects.csv` dataset from the official MET Museum source on GitHub.
2.  **Filter**: Cleans the raw data, selecting only the artworks that are relevant for our use case (i.e., public domain paintings with descriptive tags).
3.  **Enrich**: Iterates through the filtered artworks, calling the MET Collection API for each one to gather richer metadata. This data is then synthesized into a detailed text description (`enriched_text`), which is crucial for generating high-quality text embeddings.
4.  **Upload**: Pushes the final, enriched DataFrame to a Google Cloud Storage (GCS) bucket, making it readily available for ingestion into BigQuery in the next step.
---
> ### **A Note on Data Enrichment Results**
>
> You will notice that while the initial filtering step identifies a large number of potential artworks (e.g., 2,322), the final enriched dataset contains a smaller number (e.g., 81 artworks in a test run).
>
> This is an expected and intentional outcome of our robust data enrichment process. The `enrich_artwork_data` function calls the live MET Collection API for each artwork. Some of these API calls may fail due to various real-world factors, such as missing data for a specific Object ID on the API server or transient network issues.
>
> Our pipeline is designed to be resilient: instead of failing the entire process, it gracefully skips any artwork for which enrichment fails, ensuring that only complete, high-quality data is included in the final dataset. This demonstrates a practical approach to handling real-world, imperfect data sources.

In [3]:
# ===================================================================
# CELL 3: DATA PIPELINE - Fetch, Enrich, and Upload to GCS
# ===================================================================
print("--- [3/6] Starting the data processing pipeline... ---")

# --- 1. Download Source Data ---
# Download the MetObjects dataset from the official MET Museum GitHub repository.
URL = "https://media.githubusercontent.com/media/metmuseum/openaccess/master/MetObjects.csv"
print("\n--- [Step 1/4] Downloading source data from GitHub... ---")
df = pd.read_csv(URL, low_memory=False)
print(f"✅ Download complete. Loaded {len(df)} total records.")

# --- 2. Filter for Relevant Artworks ---
# Filter the dataset to include only public domain paintings that have associated tags.
print("\n--- [Step 2/4] Filtering for public domain paintings... ---")
required_columns = ['Object ID', 'Is Public Domain', 'Department', 'Title', 'Artist Display Name', 'Tags']
filtered_df = df.dropna(subset=required_columns).copy()
filtered_df = filtered_df[filtered_df['Is Public Domain'] == True]
filtered_df = filtered_df[filtered_df['Department'].str.contains('Paintings', na=False)]

# Apply the row limit defined in CELL 2 for development and testing.
initial_paintings_df = filtered_df if LIMIT_ROWS is None else filtered_df.head(LIMIT_ROWS)
print(f"✅ Filtering complete. Found {len(initial_paintings_df)} artworks to process.")

# --- 3. Enrich Data via MET Collection API ---
# For each artwork, call the MET API to get detailed metadata and construct
# a rich text description that will be used for generating embeddings.
print(f"\n--- [Step 3/4] Enriching {len(initial_paintings_df)} artworks via MET API (this may take a while)... ---")
tqdm.pandas(desc="Enriching artworks")

def enrich_artwork_data(row):
    """
    Fetches detailed data for a single artwork from the MET API and formats it
    into a structured dictionary, including a synthesized 'enriched_text' field.
    """
    object_id = row['Object ID']
    try:
        api_url = f"https://collectionapi.metmuseum.org/public/collection/v1/objects/{object_id}"
        response = requests.get(api_url, timeout=10)
        if response.status_code == 200:
            api_data = response.json()
            # Combine key metadata fields into a single, descriptive text string.
            enriched_text = (
                f"Title: {api_data.get('title', '')}. "
                f"Artist: {api_data.get('artistDisplayName', '')}. "
                f"Type: {api_data.get('objectName', '')}. "
                f"Medium: {api_data.get('medium', '')}. "
                f"Date: {api_data.get('objectDate', '')}. "
                f"Description: {api_data.get('creditLine', '')}. "
                f"Tags: {row['Tags']}"
            )
            return {
                'Object_ID': object_id,
                'Title': api_data.get('title'),
                'Artist_Display_Name': api_data.get('artistDisplayName'),
                'Link_Resource': api_data.get('primaryImageSmall') or api_data.get('primaryImage'),
                'Tags': row['Tags'],
                'enriched_text': enriched_text
            }
    except requests.exceptions.RequestException:
        # Silently ignore network-related errors (e.g., timeouts) for robustness.
        return None
    return None

# Apply the enrichment function to each row in the DataFrame using progress_apply for a visual progress bar.
enriched_results = initial_paintings_df.progress_apply(enrich_artwork_data, axis=1)
final_enriched_df = pd.DataFrame(enriched_results.dropna().tolist())
print(f"✅ Enrichment complete! Successfully processed {len(final_enriched_df)} artworks.")

# --- 4. Upload Enriched Data to GCS ---
# Upload the final DataFrame as a CSV file to the designated GCS bucket.
# This file will serve as the source for our BigQuery table in the next step.
gcs_path = f"gs://{YOUR_BUCKET_NAME}/paintings_enriched_final.csv"
print(f"\n--- [Step 4/4] Uploading enriched data to GCS at: {gcs_path} ---")

# Explicitly pass the 'credentials' object to the pandas to_csv function.
# This is crucial for authenticating with GCS when using the Service Account method.
final_enriched_df.to_csv(
    gcs_path,
    index=False,
    storage_options={'token': credentials}
)
print("✅ GCS upload complete!")
print("\n--- Data Pipeline Finished ---")

--- [3/6] Starting the data processing pipeline... ---

--- [Step 1/4] Downloading source data from GitHub... ---
✅ Download complete. Loaded 484956 total records.

--- [Step 2/4] Filtering for public domain paintings... ---
✅ Filtering complete. Found 2322 artworks to process.

--- [Step 3/4] Enriching 2322 artworks via MET API (this may take a while)... ---


Enriching artworks:   0%|          | 0/2322 [00:00<?, ?it/s]

✅ Enrichment complete! Successfully processed 83 artworks.

--- [Step 4/4] Uploading enriched data to GCS at: gs://semantic-art-explorer-20250829/paintings_enriched_final.csv ---
✅ GCS upload complete!

--- Data Pipeline Finished ---


### **CELL 4: BigQuery Setup - Create Table and AI Models**

This cell prepares the core BigQuery infrastructure for our project. It uses the data uploaded to GCS in the previous step to create the necessary tables and AI models within BigQuery. The process includes:

1.  **Dataset Creation**: Ensures a BigQuery Dataset exists to act as a container for our tables and models.
2.  **Table Ingestion**: Loads the enriched CSV data from Google Cloud Storage into a new BigQuery table named `paintings_enriched`.
3.  **Remote Model Creation**: Creates BigQuery ML (BQML) remote models that serve as a bridge to powerful Vertex AI endpoints. This enables us to perform text embedding and text generation directly within BigQuery using simple SQL commands.

---
> ⚠️ **Prerequisite: BigQuery Connection**
>
> For this cell to succeed, you must have a **BigQuery Connection for Vertex AI** named **`vertex_ai_connection_us`** created in your GCP project within the `us-central1` region.

In [4]:
# ===================================================================
# CELL 4: BIGQUERY SETUP - Create Table and AI Models
# ===================================================================
print("--- [4/6] Starting BigQuery setup... ---")

# --- 1. Define BigQuery Resource Names ---
# Use the global variables from CELL 2 to define the full names for our BigQuery resources.
dataset_id_full = f"{PROJECT_ID}.{DATASET_ID}"
table_id = f"{dataset_id_full}.paintings_enriched"
gcs_path = f"gs://{YOUR_BUCKET_NAME}/paintings_enriched_final.csv"
connection_id = f"{PROJECT_ID}.{REGION}.vertex_ai_connection_us"

# --- 2. Create BigQuery Dataset (if it doesn't exist) ---
print(f"\n--- [Step 1/3] Ensuring BigQuery dataset '{dataset_id_full}' exists... ---")
try:
    bq_client.get_dataset(dataset_id_full)
    print(f"✅ Dataset '{DATASET_ID}' already exists.")
except Exception:
    print(f"Dataset '{DATASET_ID}' not found. Creating new dataset in {REGION}...")
    dataset = bigquery.Dataset(dataset_id_full)
    dataset.location = REGION
    bq_client.create_dataset(dataset, timeout=30)
    print(f"✅ Dataset '{DATASET_ID}' created successfully.")

# --- 3. Load Data from GCS into a BigQuery Table ---
print(f"\n--- [Step 2/3] Loading data from GCS into table '{table_id}'... ---")
job_config = bigquery.LoadJobConfig(
    autodetect=True, skip_leading_rows=1,
    source_format=bigquery.SourceFormat.CSV,
    write_disposition="WRITE_TRUNCATE"
)
load_job = bq_client.load_table_from_uri(gcs_path, table_id, job_config=job_config)
load_job.result()
print("✅ Table created and data loaded successfully.")

# --- 4. Create BigQuery ML (BQML) Remote Models ---
print("\n--- [Step 3/3] Creating BQML remote models for Vertex AI... ---")

# A. Text Embedding Model
# This model will be used to convert text descriptions into numerical vectors.
text_model_query = f"""
    CREATE OR REPLACE MODEL `{dataset_id_full}.art_embedding_model`
    REMOTE WITH CONNECTION `{connection_id}`
    OPTIONS (endpoint = 'text-embedding-004');
"""
bq_client.query(text_model_query).result()
print("✅ Text embedding model ('art_embedding_model') created.")

# B. Generative (Gemini) Model for Storytelling
# This model will be used to generate creative narratives about the artworks.
story_model_query = f"""
    CREATE OR REPLACE MODEL `{dataset_id_full}.art_storytelling_model`
    REMOTE WITH CONNECTION `{connection_id}`
    OPTIONS (endpoint = 'gemini-2.0-flash-001');
"""
bq_client.query(story_model_query).result()
print("✅ Generative model ('art_storytelling_model') created.")
print("\n--- BigQuery Setup Finished ---")

--- [4/6] Starting BigQuery setup... ---

--- [Step 1/3] Ensuring BigQuery dataset 'semantic-art-explorer.art_dataset' exists... ---
✅ Dataset 'art_dataset' already exists.

--- [Step 2/3] Loading data from GCS into table 'semantic-art-explorer.art_dataset.paintings_enriched'... ---
✅ Table created and data loaded successfully.

--- [Step 3/3] Creating BQML remote models for Vertex AI... ---
✅ Text embedding model ('art_embedding_model') created.
✅ Generative model ('art_storytelling_model') created.

--- BigQuery Setup Finished ---


### **CELL 5: Feature Engineering - Generate Text Embeddings**

This cell performs the core feature engineering task of the project. It uses the `art_embedding_model` created in the previous step to convert the natural language descriptions (`enriched_text`) for each artwork into high-dimensional numerical vectors, also known as embeddings.

The process involves:

1.  **Reading** the `paintings_enriched` table.
2.  **Applying** the `ML.GENERATE_EMBEDDING` function to the text column.
3.  **Storing** the original data along with the newly generated vectors into a new table, `paintings_enriched_with_vectors`.

This final table, containing a unique vector for each artwork, is the foundation for enabling powerful semantic search capabilities in the final application.

In [5]:
# ===================================================================
# CELL 5: FEATURE ENGINEERING - Generate Text Embeddings
# ===================================================================
print("--- [5/6] Starting Feature Engineering: Generating text embeddings... ---")

# --- 1. Define Table and Model Names ---
# Define the full names for the source table, the target table, and the model
# to be used, ensuring clarity and easy maintenance.
dataset_id_full = f"{PROJECT_ID}.{DATASET_ID}"
source_table_id = f"{dataset_id_full}.paintings_enriched"
target_table_id = f"{dataset_id_full}.paintings_enriched_with_vectors"
text_model_id = f"{dataset_id_full}.art_embedding_model"

# --- 2. Construct and Execute the Embedding Generation Query ---
# This SQL query reads the source table, applies the text embedding model to each
# row, and saves the output to a new, permanent table.
create_vector_table_query = f"""
CREATE OR REPLACE TABLE `{target_table_id}` AS
SELECT
    base.*, -- Select all original columns from the source table.
    ml_generate_embedding_result AS enriched_text_embedding -- Alias the new vector column for clarity.
FROM
    ML.GENERATE_EMBEDDING(
        MODEL `{text_model_id}`,
        -- The ML.GENERATE_EMBEDDING function requires the text input column
        -- to be named 'content'. This subquery selects all columns from the
        -- source table and creates a temporary alias for our 'enriched_text'
        -- column to match this requirement.
        (SELECT *, enriched_text AS content FROM `{source_table_id}`)
    ) AS base;
"""

print(f"\n--- [Step 1/1] Generating vectors and creating table '{target_table_id}'... ---")

# Execute the query and wait for the job to complete. This can take several minutes
# depending on the number of rows being processed.
bq_client.query(create_vector_table_query).result()
print("✅ Final vector table created successfully.")
print("\n--- Feature Engineering Finished: Data is now ready for semantic search! ---")

--- [5/6] Starting Feature Engineering: Generating text embeddings... ---

--- [Step 1/1] Generating vectors and creating table 'semantic-art-explorer.art_dataset.paintings_enriched_with_vectors'... ---
✅ Final vector table created successfully.

--- Feature Engineering Finished: Data is now ready for semantic search! ---
